In [11]:
import cv2
import os
import numpy as np
import imutils

import matplotlib
import matplotlib.pyplot as plt

from optical_flow import OpticalFlow
import util

In [12]:
VIDEO_FOLDER = "./video_data/"
VIDEO = 3
FRAMES = 10
VIDEO_FILE = "{}.mp4".format(VIDEO)

## Read Video

In [17]:
def KeyPointMatch(frames):
    detector = cv2.SIFT_create()
    bf = cv2.BFMatcher()
    
    prevImg = util.drawCanvasWithImg(frames[0])
    gray1 = cv2.cvtColor(prevImg, cv2.COLOR_BGR2GRAY)
    kP1, d1 = detector.detectAndCompute(gray1, None)

    ids = 0

    for eachFrame in frames:
        eachFrame = util.drawCanvasWithImg(eachFrame)
        gray2 = cv2.cvtColor(eachFrame, cv2.COLOR_BGR2GRAY)

        kP2, d2 = detector.detectAndCompute(gray2, None)
        
        matches = bf.match(d1,d2)
        matches = sorted(matches, key=lambda x: x.distance)
        
        good_matches = matches[:80]
        

        # result = cv2.drawMatches(prevImg, kP1, eachFrame, kP2, good_matches, None, flags=2)
        # cv2.imshow(str(ids) + "drawMatch", result)

        pts1 = np.float32([kP1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        pts2 = np.float32([kP2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        
        H, status = cv2.findHomography(pts2, pts1, cv2.RANSAC, 4.0)

        result = cv2.warpPerspective(eachFrame, H, (800, 600))

        prevImg = eachFrame
        ids+=1
        cv2.imwrite("./video_data/{}/keyPointMatch_{}.jpg".format(VIDEO, str(ids)), result)
        cv2.waitKey(100)

KeyPointMatch(frames)

In [14]:
def setFrame(frames, n):
    frameIds = np.linspace(0, frames.shape[0]-1, n, dtype=np.int32)
    return frames[frameIds]

frames = []

cap = cv2.VideoCapture(VIDEO_FOLDER + VIDEO_FILE)
while cap.isOpened():
    ret, frame = cap.read()
    if ret is False:
        break
    frame = cv2.resize(frame, (480, 320))
    frame = frame[40:]
    frames.append(frame)
cap.release()


frames = np.array(frames)
frames = setFrame(frames, FRAMES)

KeyPointMatch(frames)


[[ 1.00000000e+00  2.02836266e-16 -1.64887430e-12]
 [ 1.37543159e-14  1.00000000e+00 -2.36610637e-12]
 [ 4.18371385e-17  8.68322724e-19  1.00000000e+00]]
[[ 1.00000000e+00  2.02836266e-16 -1.64887430e-12]
 [ 1.37543159e-14  1.00000000e+00 -2.36610637e-12]
 [ 4.18371385e-17  8.68322724e-19  1.00000000e+00]]
[[ 9.99931630e-01  5.07910500e-05 -3.85522462e-04]
 [-7.42914486e-05  1.00006183e+00 -1.95206132e-03]
 [-2.58928641e-07  1.89539620e-07  1.00000000e+00]]
[[ 9.99924971e-01  5.44431958e-05 -1.15133834e-04]
 [-7.95954734e-05  1.00006340e+00 -1.38479799e-03]
 [-2.74882313e-07  1.99849401e-07  1.00000000e+00]]
[[ 9.99924971e-01  5.44431958e-05 -1.15133834e-04]
 [-7.95954734e-05  1.00006340e+00 -1.38479799e-03]
 [-2.74882313e-07  1.99849401e-07  1.00000000e+00]]
[[ 9.99927715e-01  5.26419820e-05 -2.98792956e-05]
 [-7.72569732e-05  1.00006161e+00 -1.20744802e-03]
 [-2.63459477e-07  1.93369016e-07  1.00000000e+00]]
[[ 9.90594127e-01  2.47573531e-03 -8.26286841e-01]
 [-3.71412744e-03  9.9637

In [5]:
def calibOptPoints(points):
    diff = points[:, -1] - points[:, 0]
    diff_s = diff[:, 0]**2 + diff[:,1]**2
    min_id = np.argmin(diff_s)
    max_id = np.argmax(diff_s)

    diff_s -= diff_s[min_id]
    diff_s /= diff_s[max_id]

    D = diff_s.argsort()
    return points[D], diff_s[D]
    


def trackOptPoints(frames):
    OptFlw = OpticalFlow(100, FRAMES)
    prevImg = frames[0]
    canvas = np.zeros(frames[0].shape)
    
    gray = cv2.cvtColor(prevImg, cv2.COLOR_BGR2GRAY)
    points = cv2.goodFeaturesToTrack(gray, 100, 0.01, 10)
    ids = 0
    
    OptFlw.appendTrackPoint(points)
    for eachFrame in frames:
        res = OptFlw.trace(prevImg, eachFrame, drawOn=canvas)
        cv2.imwrite("./video_data/{}/{}.jpg".format(VIDEO, ids), canvas)
        prevImg = eachFrame
        ids+=1
    return np.array(OptFlw.getPoints())


points = trackOptPoints(frames)


In [15]:

def draw_tracks(frame, points, colors, until):
    for i in range(len(points)):
        for each_track in range(1, len(points[i])):
            if each_track > until:
                break
            p1, p2 = np.array(points[i][each_track - 1], dtype=np.int32), np.array(points[i][each_track], dtype=np.int32)

            thickness = int(np.sqrt(each_track)) + 1  
            itsColor = int(colors[i])
            itsColor = tuple([itsColor, itsColor, itsColor])
            
            cv2.line(frame, p1, p2, color=itsColor, thickness=thickness)


def drawStrength(frames, points):
    points, diff_s = calibOptPoints(points)
    canvas = np.zeros(frames[0].shape, dtype=np.uint8)
    s_c = (diff_s*255).astype(np.int32)
    
    ids = 0
    for eachFrame in frames:
        draw_tracks(eachFrame, points, s_c, ids)

        cv2.imshow("h2", eachFrame)
        cv2.waitKey(120)
        ids += 1

# drawStrength(frames, points)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
# trackPoints = np.transpose(np.array(trackOptPoints(frames)), (1, 0, 2)).astype(np.float32)
# l = trackPoints.shape[0]
# pts1 = np.float32([trackPoints[0][0],trackPoints[0][1],trackPoints[0][2], trackPoints[0][3]])

# canvas = np.zeros((700, 500))
# for i in range(0, l-1):
#     eachFrame = drawCanvasWithImg(frames[i])
#     pts2 = np.float32([trackPoints[i][0],trackPoints[i][1],trackPoints[i][2], trackPoints[i][3]])

#     M = cv2.getPerspectiveTransform(pts2, pts1)
#     heat = cv2.warpPerspective(eachFrame, M, (800, 500), flags=cv2.INTER_NEAREST)
    
#     cv2.imshow("0", heat)

#     cv2.waitKey(120)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [8]:
cv2.destroyAllWindows()